In [4]:
import base64
import os

import cv2
import numpy as np
import requests
import pickle

url = 'http://localhost:8080/api/gethog'



class_brand = ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce',
               'Swift', 'Tata Safari', 'Toyota Innova']

#img -> base64
def img2HOG(img):
    v,buffer = cv2.imencode('.jpg',img)
    img_str = base64.b64encode(buffer)
    data = "image data,"+str.split(str(img_str),"'")[1]
    response = requests.get(url,json={"img":data})
    
    return response.json()

#read pic
def readData(path):
    response = []
    for folder in os.listdir(path):
        for img_name in os.listdir(path+'/'+folder):
            img_path = path+'/'+folder+'/'+img_name
            img = cv2.imread(img_path)
            res = img2HOG(img)
            hog = list(res["HOG"])
            hog.append(class_brand.index(folder))
            response.append(hog)
    
    return response

In [5]:
path = r'C:\Users\lalik\Downloads\train'
datasetHOG = readData(path)

In [6]:
write_path = "imgHOG.pkl"
pickle.dump(datasetHOG,open(write_path,"wb"))

In [7]:
#Load data from pkl
imgHog = pickle.load(open('imgHog.pkl','rb'))
# print(len(imgHog))

In [8]:
#change list to array
imgHog_np = np.array(imgHog)
x_train = imgHog_np[:,0:-1]
y_train = imgHog_np[:,-1]

In [9]:
#check size
print(x_train.shape)
print(y_train.shape)

(3352, 8100)
(3352,)


In [10]:
#read pic test
pathtest = r'C:\Users\lalik\Downloads\test'
dataset_test = readData(pathtest)

In [11]:
#save data
write_path = "imgHOG_test.pkl"
pickle.dump(dataset_test,open(write_path,"wb"))

In [12]:
#load data
dataset_test = pickle.load(open('imgHOG_test.pkl','rb'))
print(len(dataset_test))

813


In [13]:
#list to array
dataset_test_np = np.array(dataset_test)
x_test = dataset_test_np[:,0:-1]
y_test = dataset_test_np[:,-1]

In [14]:
#Tran model part
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [15]:
#predict part
from sklearn import metrics

y_pred = clf.predict(x_test)
acc = metrics.accuracy_score(y_test,y_pred)
cofus_mect = metrics.confusion_matrix(y_test,y_pred)
print("accuracy = ",acc*100)
print('Confusion Mectrix :\n',cofus_mect)

accuracy =  47.23247232472325
Confusion Mectrix :
 [[ 74  20  15  21  20  25  24]
 [ 12  38   3   1   1   4   8]
 [ 13   1  23   9   4   9  16]
 [ 23   3   7  16   6   8  11]
 [ 11   9   6   6  46   4  20]
 [ 15   6   1   7   4  66   7]
 [ 24   8   8  12  10   7 121]]


In [16]:
#save model
path_model = 'cls_carModel.pkl'
pickle.dump(clf,open(path_model,'wb'))


In [17]:
#model XGBOOST
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(x_train,y_train)

c:\1-2566\AICLASS\traincarmodel\env\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob',
              predictor=None, ...)

In [18]:
#predict Xgboost
y_pred_xgb = xgb.predict(x_test)
acc_xgb = metrics.accuracy_score(y_test,y_pred_xgb)
cofus_mect_xgb = metrics.confusion_matrix(y_test,y_pred_xgb)
print("accuracy = ",acc_xgb*100)
print('Confusion Mectrix :\n',cofus_mect_xgb)

accuracy =  65.68265682656826
Confusion Mectrix :
 [[162   3   2   5   5   8  14]
 [  7  41   2   2   3   1  11]
 [  9   0  37   0   5   3  21]
 [ 44   2   3  13   3   1   8]
 [ 14   0   2   2  51   3  30]
 [ 15   1   1   0   3  78   8]
 [ 25   0   1   4   4   4 152]]


In [19]:
#save model
path_model = 'model_XGB.pkl'
pickle.dump(xgb,open(path_model,'wb'))